In [1]:
# Load packages
import nltk # imports the natural language toolkit
nltk.download('punkt')
nltk.download('stopwords')
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import string
import re
from collections import Counter
from nltk.stem import PorterStemmer
from nltk.util import ngrams
#from pylab import rcParamsb
from sklearn.feature_extraction.text import CountVectorizer
import plotly.express as px
import nltk
from wordcloud import WordCloud
import os

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sapmn3\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sapmn3\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from data_exploring.data_exploring_functions import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sapmn3\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#File names and base path.
base_datapath = 'C:\\Users\\sapmn3\\Database\\OFFCORSS\\data\\OFFCORSS_NPS'
file_list = ['2019-1-2019-2-2019-3_satisfaction-ratings',
            '2019-4-2019-5_satisfaction-ratings',
            '2019-6-2019-7_satisfaction-ratings',
            '2019-8-2019-9_satisfaction-ratings',
            '2019-10-2019-11_satisfaction-ratings',
            '2019-12_satisfaction-ratings',
            '2020-1-2020-2_satisfaction-ratings',
            '2020-3-2020-4_satisfaction-ratings',
            '2020-5-2020-6_satisfaction-ratings',
            '2020-7-2020-8_satisfaction-ratings',
            '2020-9-2020-10_satisfaction-ratings',
            'Satisfaction_Ratings',
            'satisfaction-ratings-2020-10-08',
            'satisfaction-ratings-2020-10-09',
            ]

data_nps = []
for cur_file in file_list:
    cur_path = os.path.join(base_datapath, cur_file + '.csv')
    cur_data = pd.read_csv(cur_path)
    cur_data['Survey Date'] = pd.to_datetime(cur_data['Survey Date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    cur_data['Survey Date'] = cur_data['Survey Date'].dt.date
    if len(data_nps) == 0:
        data_nps = cur_data
    else:
        data_nps = pd.concat([data_nps, cur_data], axis=0)


In [4]:
#Load the Speller
spell = Speller(lang='es')

#Perform spell correction and use lower letters to the comment column
data_nps_corrected = spell_correction(data_nps, 'Comment', 'processed comment', spell)
data_nps_corrected

working on it!


,Requester,User Id,Email,Ticket Id,Brand,Group,Assignee,Satisfaction,Comment,Survey Date,processed comment
0,marli rojas,1356922323,marilin2906@hotmail.com,434654,OFFCORSS,Soporte OFFCORSS,Jurídica,good,excelente,2019-03-31,excelente
1,marli rojas,1356922323,marilin2906@hotmail.com,434654,OFFCORSS,Soporte OFFCORSS,Jurídica,good,NaN,2019-03-31,NaN
2,luchycarmonamedina,378832242231,luchycarmonamedina@gmail.com,436246,OFFCORSS,Soporte OFFCORSS,CTS Transporte,bad,NaN,2019-03-31,NaN
3,Idali Gamboa,374617743451,laila1709_@hotmail.com,435404,OFFCORSS,Soporte OFFCORSS,Devoluciones,good,NaN,2019-03-30,NaN
4,Ingrid Tatiana Muñoz Rojas,378402209612,intamuro@hotmail.com,434377,OFFCORSS,Venta Directa,Interés Comercial VD,good,NaN,2019-03-30,NaN
...,...,...,...,...,...,...,...,...,...,...,...
6387,Dina.p16,360797207046,dina.p16@hotmail.com,529256,OFFCORSS,Soporte OFFCORSS,CTS Transporte,bad,No me llego el pedido,2020-05-01,no me llego el pedido
6388,ADRANA BURGOS,409831429,adrianamaria03@hotmail.com,531331,OFFCORSS,Tienda Virtual,Servicio al cliente TV,bad,"Malo , ustedes publican pantalón o chaqueta y...",2020-05-01,"malo , ustedes publican pantalón o chaqueta y ..."
6389,ADRANA BURGOS,409831429,adrianamaria03@hotmail.com,531331,OFFCORSS,Tienda Virtual,Servicio al cliente TV,bad,"Malo , ustedes publican pantalón o chaqueta y...",2020-05-01,"malo , ustedes publican pantalón o chaqueta y ..."
6390,Devolución de producto,397315344372,lorenamoreno1423@gmail.com,530275,OFFCORSS,Call Center,Call Center,good,NaN,2020-05-01,NaN


In [5]:
# Consolidate responses for both types:

id_good = data_nps_corrected['Satisfaction'].isnull() & (data_nps_corrected['Satisfaction'] == 'good')
id_bad  = data_nps_corrected['Satisfaction'].isnull() & (data_nps_corrected['Satisfaction'] == 'bad')

data_sentiment_standard = data_nps_corrected.copy()

data_sentiment_standard.drop_duplicates(subset=['Ticket Id', 'processed comment'], inplace = True, keep = 'first')
data_sentiment_standard

,Requester,User Id,Email,Ticket Id,Brand,Group,Assignee,Satisfaction,Comment,Survey Date,processed comment
0,marli rojas,1356922323,marilin2906@hotmail.com,434654,OFFCORSS,Soporte OFFCORSS,Jurídica,good,excelente,2019-03-31,excelente
1,marli rojas,1356922323,marilin2906@hotmail.com,434654,OFFCORSS,Soporte OFFCORSS,Jurídica,good,NaN,2019-03-31,NaN
2,luchycarmonamedina,378832242231,luchycarmonamedina@gmail.com,436246,OFFCORSS,Soporte OFFCORSS,CTS Transporte,bad,NaN,2019-03-31,NaN
3,Idali Gamboa,374617743451,laila1709_@hotmail.com,435404,OFFCORSS,Soporte OFFCORSS,Devoluciones,good,NaN,2019-03-30,NaN
4,Ingrid Tatiana Muñoz Rojas,378402209612,intamuro@hotmail.com,434377,OFFCORSS,Venta Directa,Interés Comercial VD,good,NaN,2019-03-30,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2413,carlos arias barrera,402216692612,carlosariasbarrera@yahoo.es,605721,OFFCORSS,Call Center,Call Center,bad,Buenos dias dias señores Offcorss desde el dia...,2020-09-01,buenos dias dias señores offcorss desde el dia...
2414,ktrin.moya,402594757052,ktrin.moya@hotmail.com,609823,OFFCORSS,Soporte OFFCORSS,CTS Transporte,good,NaN,2020-09-01,NaN
2415,Devolución de producto,372913262211,miormo19-83@hotmail.com,609793,OFFCORSS,Soporte OFFCORSS,CTS Transporte,good,siempre ha sido bueno,2020-09-01,siempre ha sido bueno
2416,Zona620vc,361034608766,zona620vc@offcorss.com,610864,OFFCORSS,Venta Directa,Interés Comercial VD,good,Buenos dias....\r\n\r\nsuper muy rapido el pro...,2020-09-01,buenos dias .... super muy rapido el proceso m...


In [8]:
# Reorganize elements by date
data_sentiment_standard['Survey Date'] = pd.to_datetime(data_sentiment_standard['Survey Date'])
data_sentiment_standard = data_sentiment_standard.sort_values('Survey Date').reset_index(drop = True) 
#data_sentiment_standard = data_sentiment_standard.drop(columns=['level_0'])
data_sentiment_standard

,Requester,User Id,Email,Ticket Id,Brand,Group,Assignee,Satisfaction,Comment,Survey Date,processed comment
0,alejo.datenshi,374298422771,alejo.datenshi@gmail.com,416857,OFFCORSS,Call Center,VENTAS TIENDA VIRTUAL,good,NaN,2019-01-01,NaN
1,Carime Suarez,1871267083,luzkarinasuarez1980@hotmail.com,421812,OFFCORSS,Tienda Virtual,Servicio al cliente TV,good,NaN,2019-01-01,NaN
2,marketsevenstar,374852675191,marketsevenstar@gmail.com,421796,OFFCORSS,Tienda Virtual,Servicio al cliente TV,bad,ya no deseo hacer el pedido \r\nes la tercera ...,2019-01-01,ya no deseo hacer el pedido es la tercera vez ...
3,Paola Bermudez Motoa,374404056191,paolabermudezmotoa@gmail.com,418196,OFFCORSS,Call Center,Call Center,bad,Me perdieron como cliente pesimo servicio al c...,2019-01-01,me perdieron como cliente pésimo servicio al c...
4,Devolución de producto,7576067323,tmargarita-01@hotmail.com,417254,OFFCORSS,Call Center,VENTAS TIENDA VIRTUAL,good,"Muy buena calidad, soporte y atencion al clie...",2019-01-02,"muy buena calidad , soporte y atencion al clie..."
...,...,...,...,...,...,...,...,...,...,...,...
28065,Daniela Sanchez Osorio,404360644371,danielita-422@hotmail.com,627304,OFFCORSS,Venta Directa,Interés Comercial VD,good,NaN,2020-10-09,NaN
28066,lijovima22,404359464492,lijovima22@gmail.com,627535,OFFCORSS,Soporte OFFCORSS,CTS Transporte,good,Que al momento de la cancelación del pedido se...,2020-10-09,que al momento de la cancelación del pedido se...
28067,mdelaespriellav,404010320731,mdelaespriellav@gmail.com,629822,OFFCORSS,Call Center,Call Center,good,EXCELENTE,2020-10-09,excelente
28068,Angela Castrillon,7303733323,avivianac@gmail.com,630060,OFFCORSS,Call Center,Call Center,good,NaN,2020-10-09,NaN


In [9]:
# Save processed files
save_datapath = 'C:\\Users\\sapmn3\\Database\\OFFCORSS\\data\\data_clean'
save_file = 'offcorss_sentiment'
save_path = os.path.join(save_datapath, save_file + '.csv')

data_sentiment_standard.to_csv(save_path, encoding='utf-8-sig')
    